## 1.	Using the data from delta and order data frame, create a new data frame where each price point is updated using information from the order


For example: For a given price point, if a particular event is update and the order was buy of size 200, we can subtract the size of the order book (snapshot) at that particular price point for sell by 200. Repeat this for all the orders. Make sure to include the timestamp and crossseq.

## 2.	Create a function that will take the timestamp from delta data frame to update the snapshot sequentially.
For example: At time t, we have an event type ‘update’ for price point X. If the price point is sell, then we subtract and add if its buy. Do the same for ‘delete’ and ‘insert’

## 3.	Combine the data frame from 1 and 2, and arrange them by timestamp for the normalized data

In [303]:
import pandas as pd
import matplotlib.pyplot as plt

In [270]:
#creating a dataframe snapshot
df_snapshot = pd.read_csv('snapshot.csv')
df_snapshot

,price,symbol,id,side,size,timestamp_e6,cross_seq
0,47660.0,BTCUSD,476600000,Buy,40,1640274379047154,4775819907
1,47676.5,BTCUSD,476765000,Buy,100,1640274379047154,4775819907
2,47680.0,BTCUSD,476800000,Buy,40,1640274379047154,4775819907
3,47680.5,BTCUSD,476805000,Buy,8000,1640274379047154,4775819907
4,47700.0,BTCUSD,477000000,Buy,40,1640274379047154,4775819907
...,...,...,...,...,...,...,...
395,49304.0,BTCUSD,493040000,Sell,8398,1640274379047154,4775819907
396,49305.5,BTCUSD,493055000,Sell,87139,1640274379047154,4775819907
397,49311.0,BTCUSD,493110000,Sell,59084,1640274379047154,4775819907
398,49314.0,BTCUSD,493140000,Sell,7,1640274379047154,4775819907


In [311]:
df_snapshot.id.nunique()

400

In [300]:
df_snapshot.price.nunique()

400

In [271]:
df_snapshot.cross_seq.nunique()

1

In [276]:
#creating a delta snapshot
df_delta = pd.read_csv('delta.csv')
df_delta

,event_type,price,symbol,id,side,size,cross_seq,timestamp_e6
0,delete,47660.0,BTCUSD,476600000,Buy,NaN,4775819908,1640274379247565
1,insert,48701.0,BTCUSD,487010000,Buy,902.0,4775819908,1640274379247565
2,update,48763.0,BTCUSD,487630000,Sell,57820.0,4775819912,1640274380247486
3,update,48762.5,BTCUSD,487625000,Sell,77610.0,4775819912,1640274380247486
4,update,48744.0,BTCUSD,487440000,Buy,162010.0,4775819912,1640274380247486
...,...,...,...,...,...,...,...,...
912,update,48724.0,BTCUSD,487240000,Buy,132060.0,4775820817,1640274440147770
913,insert,49297.5,BTCUSD,492975000,Sell,55830.0,4775820817,1640274440147770
914,delete,47773.5,BTCUSD,477735000,Buy,NaN,4775820819,1640274440247826
915,update,48794.0,BTCUSD,487940000,Sell,1011.0,4775820819,1640274440247826


In [309]:
df_delta.price.nunique()

248

In [277]:
df_delta.cross_seq.nunique()

135

In [278]:
#creating a trade snapshot
df_trade = pd.read_csv('trade.csv')
df_trade

,timestamp,trade_time_ms,symbol,trade_id,side,size,price,cross_seq,tick_direction
0,2021-12-23T15:46:22.000Z,1640274382818,BTCUSD,216bcf4e-895f-566c-985c-923af9c63fcf,Buy,200,48760.0,4775819918,ZeroPlusTick
1,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,94487830-2202-5bad-a9fc-dd462480863f,Sell,6,48759.5,4775820093,MinusTick
2,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,43ae8892-477f-5e88-b795-bc0e4abef966,Sell,10,48759.5,4775820093,ZeroMinusTick
3,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,7df7b72b-5559-5bd0-a82a-84e028b0c515,Sell,20,48750.0,4775820093,MinusTick
4,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,28cf0587-6a95-56d0-99fe-be2c3a5c8849,Sell,20,48740.0,4775820093,MinusTick
5,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,1b9d3777-4d2e-56cb-b326-8223f71f86ec,Sell,244,48738.0,4775820093,MinusTick
6,2021-12-23T15:46:41.000Z,1640274401826,BTCUSD,1df93f97-a369-5f68-a6b8-f6ccae6772cb,Sell,2700,48738.0,4775820128,ZeroMinusTick
7,2021-12-23T15:46:43.000Z,1640274403520,BTCUSD,f467ca9c-8075-5880-8e77-a0d438281010,Buy,300,48739.0,4775820141,PlusTick
8,2021-12-23T15:46:52.000Z,1640274412581,BTCUSD,2c159acb-12cf-575a-a6a6-d7ee44f9184f,Sell,19,48740.5,4775820381,PlusTick
9,2021-12-23T15:46:54.000Z,1640274414827,BTCUSD,5c20d845-5edd-5598-ab2b-72ed3125354d,Sell,23,48744.5,4775820437,PlusTick


In [279]:
df_trade.cross_seq.nunique()

10

In [280]:
#creating a snapshot df with no insert - to stop it from influencing the order filtering i.e. avoiding market orders
df_delta_no_insert = df_delta[df_delta['event_type'] != 'insert']
df_delta_no_insert

,event_type,price,symbol,id,side,size,cross_seq,timestamp_e6
0,delete,47660.0,BTCUSD,476600000,Buy,NaN,4775819908,1640274379247565
2,update,48763.0,BTCUSD,487630000,Sell,57820.0,4775819912,1640274380247486
3,update,48762.5,BTCUSD,487625000,Sell,77610.0,4775819912,1640274380247486
4,update,48744.0,BTCUSD,487440000,Buy,162010.0,4775819912,1640274380247486
5,update,48762.0,BTCUSD,487620000,Sell,52370.0,4775819912,1640274380247486
...,...,...,...,...,...,...,...,...
910,update,48748.0,BTCUSD,487480000,Sell,140860.0,4775820817,1640274440147770
911,update,48747.5,BTCUSD,487475000,Sell,153700.0,4775820817,1640274440147770
912,update,48724.0,BTCUSD,487240000,Buy,132060.0,4775820817,1640274440147770
914,delete,47773.5,BTCUSD,477735000,Buy,NaN,4775820819,1640274440247826


In [316]:
#data frame connecting market order and delta
df_delta_trade = pd.merge(df_delta, df_trade, on=["cross_seq", "price"])
df_delta_trade

,event_type,price,symbol_x,id,side_x,size_x,cross_seq,timestamp_e6,timestamp,trade_time_ms,symbol_y,trade_id,side_y,size_y,tick_direction
0,update,48760.0,BTCUSD,487600000,Sell,78150.0,4775819918,1640274382847562,2021-12-23T15:46:22.000Z,1640274382818,BTCUSD,216bcf4e-895f-566c-985c-923af9c63fcf,Buy,200,ZeroPlusTick
1,delete,48759.5,BTCUSD,487595000,Buy,NaN,4775820093,1640274400347496,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,94487830-2202-5bad-a9fc-dd462480863f,Sell,6,MinusTick
2,delete,48759.5,BTCUSD,487595000,Buy,NaN,4775820093,1640274400347496,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,43ae8892-477f-5e88-b795-bc0e4abef966,Sell,10,ZeroMinusTick
3,delete,48740.0,BTCUSD,487400000,Buy,NaN,4775820093,1640274400347496,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,28cf0587-6a95-56d0-99fe-be2c3a5c8849,Sell,20,MinusTick
4,delete,48750.0,BTCUSD,487500000,Buy,NaN,4775820093,1640274400347496,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,7df7b72b-5559-5bd0-a82a-84e028b0c515,Sell,20,MinusTick
5,update,48738.0,BTCUSD,487380000,Buy,11756.0,4775820093,1640274400347496,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,1b9d3777-4d2e-56cb-b326-8223f71f86ec,Sell,244,MinusTick
6,update,48738.0,BTCUSD,487380000,Buy,9056.0,4775820128,1640274401847574,2021-12-23T15:46:41.000Z,1640274401826,BTCUSD,1df93f97-a369-5f68-a6b8-f6ccae6772cb,Sell,2700,ZeroMinusTick
7,update,48739.0,BTCUSD,487390000,Sell,77340.0,4775820141,1640274403547485,2021-12-23T15:46:43.000Z,1640274403520,BTCUSD,f467ca9c-8075-5880-8e77-a0d438281010,Buy,300,PlusTick
8,update,48740.5,BTCUSD,487405000,Buy,76401.0,4775820381,1640274412647751,2021-12-23T15:46:52.000Z,1640274412581,BTCUSD,2c159acb-12cf-575a-a6a6-d7ee44f9184f,Sell,19,PlusTick
9,update,48744.5,BTCUSD,487445000,Buy,118447.0,4775820437,1640274414847585,2021-12-23T15:46:54.000Z,1640274414827,BTCUSD,5c20d845-5edd-5598-ab2b-72ed3125354d,Sell,23,PlusTick
